# Data Structures
`Cross-sectional` data is collected at a single point in time, capturing information across various subjects to analyze patterns and relationships among variables at that moment.

`Time-series data` is collected over different time periods for the same subject, allowing the tracking of changes and trends over time.

`Panel (or longitudinal) data` combines cross-sectional and time-series elements by collecting data on the same subjects across multiple time periods, enabling analysis of both time-based changes and cross-sectional differences.

## Panel Data

### Download data

In [2]:
import pandas as pd
import urllib.parse

# This script demonstrates how to download a CSV file from the Statistics Canada website
# using the downloadDbLoadingData-nonTraduit.action endpoint.
base_url = "https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action"
pid = "1410028701"
latestN =  ""
startDate = "20190101"
endDate = ""
csvLocale = "en"
selected_members = "[[1,9],[1,2,3,4,5,6,7,8,9],[1,2,3],[1,2,6,7],[1],[1,2]]"

#encode selected members
encoded_selected_members = urllib.parse.quote(selected_members,safe='')

# Construct the full URL
url = "{}?pid={}&latestN={}&startDate={}&endDate={}&csvLocale=en{}\
&selectedMembers={}".format(base_url,pid,latestN,startDate,endDate,
csvLocale,encoded_selected_members)

# Read the CSV data into a DataFrame
df_1 = pd.read_csv(url)

df_1.tail()

,REF_DATE,GEO,DGUID,Labour force characteristics,Gender,Age group,Statistics,Data type,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
30947,2024-09,Saskatchewan,2021A000247,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066588,9.9.3.7.1.2,33.1,NaN,NaN,NaN,1
30948,2024-10,Saskatchewan,2021A000247,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066588,9.9.3.7.1.2,32.3,NaN,NaN,NaN,1
30949,2024-11,Saskatchewan,2021A000247,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066588,9.9.3.7.1.2,32.1,NaN,NaN,NaN,1
30950,2024-12,Saskatchewan,2021A000247,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066588,9.9.3.7.1.2,32.1,NaN,NaN,NaN,1
30951,2025-01,Saskatchewan,2021A000247,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066588,9.9.3.7.1.2,31.1,NaN,NaN,NaN,1


### Remove extra columns

In [91]:
# drop extra columns
columns_to_drop = ['DGUID', 'COORDINATE', 'Statistics', 'UOM', 'SCALAR_FACTOR',
'UOM_ID','SCALAR_ID','VECTOR', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']

df_2 = df_1.drop(columns=[col for col in columns_to_drop if col in df_1.columns], axis=1)

# Display the first few rows of the DataFrame
df_2.tail()

,REF_DATE,GEO,Labour force characteristics,Gender,Age group,Data type,VALUE
30947,2024-09,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,33.1
30948,2024-10,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,32.3
30949,2024-11,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,32.1
30950,2024-12,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,32.1
30951,2025-01,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,31.1


### Add date index

In [92]:
# convert the 'REF_DATE' column to a datetime object
df_2['REF_DATE']=pd.to_datetime(df_2['REF_DATE'], format='%Y-%m')

# set the 'REF_DATE' column as the index
df_2.set_index('REF_DATE', inplace=True)

# convert the index to a period index
df_2.index = df_2.index.to_period('M')

df_2.tail()


,GEO,Labour force characteristics,Gender,Age group,Data type,VALUE
REF_DATE,,,,,,
2024-09,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,33.1
2024-10,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,32.3
2024-11,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,32.1
2024-12,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,32.1
2025-01,Saskatchewan,Employment rate,Women+,55 years and over,Unadjusted,31.1


## Time Series
Each column in the following table is a time series

In [95]:
# pivot from long to wide format
df_3 = df_2.pivot_table(index=['REF_DATE'],
columns=['Data type','GEO','Gender','Age group','Labour force characteristics'], 
values='VALUE')

df_3.tail()

Data type                    Seasonally adjusted                  \
GEO                                       Canada                   
Gender                                      Men+                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE                                                           
2024-09                                   1368.0            53.1   
2024-10                                   1379.2            53.4   
2024-11                                   1379.7            53.3   
2024-12                                   1375.1            53.0   
2025-01                                   1399.7            53.8   

Data type                                                       \
GEO                                                              
Gender                                                           
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE                                                         
2024-09                                     780.0       1615.2   
2024-10                                     795.5       1615.8   
2024-11                                     797.6       1635.3   
2024-12                                     800.7       1634.9   
2025-01                                     831.6       1637.7   

Data type                                                             \
GEO                                                                    
Gender                                                                 
Age group                                                              
Labour force characteristics Part-time employment Participation rate   
REF_DATE                                                               
2024-09                                     588.0               62.7   
2024-10                                     583.7               62.5   
2024-11                                     582.0               63.1   
2024-12                                     574.5               63.0   
2025-01                                     568.2               63.0   

Data type                                                               \
GEO                                                                      
Gender                                                                   
Age group                                                                
Labour force characteristics Population Unemployment Unemployment rate   
REF_DATE                                                                 
2024-09                          2577.4        247.2              15.3   
2024-10                          2584.8        236.6              14.6   
2024-11                          2590.2        255.6              15.6   
2024-12                          2595.4        259.8              15.9   
2025-01                          2599.4        237.9              14.5   

Data type                                       ...        Unadjusted  \
GEO                                             ...      Saskatchewan   
Gender                                          ...            Women+   
Age group                    15 years and over  ...    25 to 54 years   
Labour force characteristics        Employment  ... Unemployment rate   
REF_DATE                                        ...                     
2024-09                                10950.4  ...               4.7   
2024-10                                10964.1  ...               4.1   
2024-11                                10998.7  ...               3.3   
2024-12                                11059.3  ...               4.2   
2025-01                                11091.2  ...               3.8   

Data type                                                       \
GEO                                           

In [102]:
# for convenience
df = df_3

In [103]:
# columns is a multiindex
print(type(df.columns),'\n')
for i in range(0, len(df.columns.names)):
    print(f'Level {i}: {df.columns.levels[i]}\n')

<class 'pandas.core.indexes.multi.MultiIndex'> 

Level 0: Index(['Seasonally adjusted', 'Unadjusted'], dtype='object', name='Data type')

Level 1: Index(['Canada', 'Saskatchewan'], dtype='object', name='GEO')

Level 2: Index(['Men+', 'Total - Gender', 'Women+'], dtype='object', name='Gender')

Level 3: Index(['15 to 24 years', '15 years and over', '25 to 54 years',
       '55 years and over'],
      dtype='object', name='Age group')

Level 4: Index(['Employment', 'Employment rate', 'Full-time employment', 'Labour force',
       'Part-time employment', 'Participation rate', 'Population',
       'Unemployment', 'Unemployment rate'],
      dtype='object', name='Labour force characteristics')



## Reshaping

### Stack

In [99]:
#.stack() rotates the lowest level of the column MultiIndex to the row index
df.stack().tail()


C:\Users\mehdi\AppData\Local\Temp\ipykernel_10408\2996014824.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack().tail()


Data type                             Seasonally adjusted                    \
GEO                                                Canada                     
Gender                                               Men+                     
Age group                                  15 to 24 years 15 years and over   
REF_DATE Labour force characteristics                                         
2025-01  Part-time employment                       568.2            1411.4   
         Participation rate                          63.0              69.7   
         Population                                2599.4           17081.4   
         Unemployment                               237.9             810.2   
         Unemployment rate                           14.5               6.8   

Data type                                                               \
GEO                                                                      
Gender                                                                   
Age group                             25 to 54 years 55 years and over   
REF_DATE Labour force characteristics                                    
2025-01  Part-time employment                  438.2               NaN   
         Participation rate                     91.9              41.6   
         Population                           8417.2            6064.8   
         Unemployment                          433.9             138.4   
         Unemployment rate                       5.6               5.5   

Data type                                                               \
GEO                                                                      
Gender                                Total - Gender                     
Age group                             15 to 24 years 15 years and over   
REF_DATE Labour force characteristics                                    
2025-01  Part-time employment                 1328.5            3807.5   
         Participation rate                     63.1              65.5   
         Population                           4999.6           34340.8   
         Unemployment                          429.8            1490.6   
         Unemployment rate                      13.6               6.6   

Data type                                                               \
GEO                                                                      
Gender                                                                   
Age group                             25 to 54 years 55 years and over   
REF_DATE Labour force characteristics                                    
2025-01  Part-time employment                 1475.8            1003.2   
         Participation rate                     88.6              36.2   
         Population                          16632.1           12709.0   
         Unemployment                          828.3             232.5   
         Unemployment rate                       5.6               5.1   

Data type                                                               ...  \
GEO                                                                     ...   
Gender                                        Women+                    ...   
Age group                             15 to 24 years 15 years and over  ...   
REF_DATE Labour force characteristics                                   ...   
2025-01  Part-time employment                  760.3            2396.1  ...   
         Participation rate                     63.2              61.3  ...   
         Population                           2400.3           17259.4  ...   
         Unemployment                          191.9             680.3  ...   
         Unemployment rate                      12.7               6.4  ...   

Data type                                 Unadjusted                    \
GEO                                     Saskatchewan                     
Gender                                         

In [70]:
#Stacking will move the lowest level of the column MultiIndex to the
# row index and creates a multilevel row index
df.stack().index

C:\Users\mehdi\AppData\Local\Temp\ipykernel_10408\1171119008.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack().index


MultiIndex([('2019-01-01',           'Employment'),
            ('2019-01-01',      'Employment rate'),
            ('2019-01-01', 'Full-time employment'),
            ('2019-01-01',         'Labour force'),
            ('2019-01-01', 'Part-time employment'),
            ('2019-01-01',   'Participation rate'),
            ('2019-01-01',           'Population'),
            ('2019-01-01',         'Unemployment'),
            ('2019-01-01',    'Unemployment rate'),
            ('2019-02-01',           'Employment'),
            ...
            ('2024-12-01',    'Unemployment rate'),
            ('2025-01-01',           'Employment'),
            ('2025-01-01',      'Employment rate'),
            ('2025-01-01', 'Full-time employment'),
            ('2025-01-01',         'Labour force'),
            ('2025-01-01', 'Part-time employment'),
            ('2025-01-01',   'Participation rate'),
            ('2025-01-01',           'Population'),
            ('2025-01-01',         'Unemployment

In [73]:
# We can also pass in an argument to select the level we would like to stack
df.stack(level=2).tail()

C:\Users\mehdi\AppData\Local\Temp\ipykernel_10408\783117807.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack(level=2).tail()


Data type                    Seasonally adjusted                  \
GEO                                       Canada                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE   Gender                                                  
2024-12-01 Total - Gender                 2693.3            53.9   
           Women+                         1318.2            55.0   
2025-01-01 Men+                           1399.7            53.8   
           Total - Gender                 2723.8            54.5   
           Women+                         1324.0            55.2   

Data type                                                       \
GEO                                                              
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE   Gender                                                
2024-12-01 Total - Gender                  1364.0       3137.6   
           Women+                           563.4       1502.7   
2025-01-01 Men+                             831.6       1637.7   
           Total - Gender                  1395.3       3153.6   
           Women+                           563.7       1515.9   

Data type                                                             \
GEO                                                                    
Age group                                                              
Labour force characteristics Part-time employment Participation rate   
REF_DATE   Gender                                                      
2024-12-01 Total - Gender                  1329.2               62.8   
           Women+                           754.8               62.7   
2025-01-01 Men+                             568.2               63.0   
           Total - Gender                  1328.5               63.1   
           Women+                           760.3               63.2   

Data type                                                               \
GEO                                                                      
Age group                                                                
Labour force characteristics Population Unemployment Unemployment rate   
REF_DATE   Gender                                                        
2024-12-01 Total - Gender        4993.3        444.4              14.2   
           Women+                2397.9        184.6              12.3   
2025-01-01 Men+                  2599.4        237.9              14.5   
           Total - Gender        4999.6        429.8              13.6   
           Women+                2400.3        191.9              12.7   

Data type                                       ...        Unadjusted  \
GEO                                             ...      Saskatchewan   
Age group                    15 years and over  ...    25 to 54 years   
Labour force characteristics        Employment  ... Unemployment rate   
REF_DATE   Gender                               ...                     
2024-12-01 Total - Gender              20917.4  ...               4.5   
           Women+                       9858.0  ...               4.2   
2025-01-01 Men+                        11091.2  ...               5.0   
           Total - Gender              20993.4  ...               4.4   
           Women+                       9902.2  ...               3.8   

Data type                                                       \
GEO                                                              
Age group                    55 years and over                   
Labour force characteristics        Employment Employment rate   
REF_DATE   Gender                                                
2024-12-01 Total - Gender                127.9            37.6   
           Women+                         56.4            32.1   
202

In [74]:
# multiple levels can be stacked at once
df.stack(level=[1,2]).tail()

C:\Users\mehdi\AppData\Local\Temp\ipykernel_10408\125885140.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack(level=[1,2]).tail()


Data type                              Seasonally adjusted                  \
Age group                                   15 to 24 years                   
Labour force characteristics                    Employment Employment rate   
REF_DATE   GEO          Gender                                               
2025-01-01 Canada       Total - Gender              2723.8            54.5   
                        Women+                      1324.0            55.2   
           Saskatchewan Men+                          42.0            54.1   
                        Total - Gender                81.4            54.7   
                        Women+                        39.4            55.4   

Data type                                                                 \
Age group                                                                  
Labour force characteristics           Full-time employment Labour force   
REF_DATE   GEO          Gender                                             
2025-01-01 Canada       Total - Gender               1395.3       3153.6   
                        Women+                        563.7       1515.9   
           Saskatchewan Men+                           30.4         48.6   
                        Total - Gender                 49.9         92.2   
                        Women+                         19.5         43.5   

Data type                                                    \
Age group                                                     
Labour force characteristics           Part-time employment   
REF_DATE   GEO          Gender                                
2025-01-01 Canada       Total - Gender               1328.5   
                        Women+                        760.3   
           Saskatchewan Men+                           11.6   
                        Total - Gender                 31.5   
                        Women+                         19.9   

Data type                                                             \
Age group                                                              
Labour force characteristics           Participation rate Population   
REF_DATE   GEO          Gender                                         
2025-01-01 Canada       Total - Gender               63.1     4999.6   
                        Women+                       63.2     2400.3   
           Saskatchewan Men+                         62.6       77.6   
                        Total - Gender               62.0      148.7   
                        Women+                       61.2       71.1   

Data type                                                              \
Age group                                                               
Labour force characteristics           Unemployment Unemployment rate   
REF_DATE   GEO          Gender                                          
2025-01-01 Canada       Total - Gender        429.8              13.6   
                        Women+                191.9              12.7   
           Saskatchewan Men+                    6.7              13.8   
                        Total - Gender         10.8              11.7   
                        Women+                  4.1               9.4   

Data type                                                 ...  \
Age group                              15 years and over  ...   
Labour force characteristics                  Employment  ...   
REF_DATE   GEO          Gender                            ...   
2025-01-01 Canada       Total - Gender           20993.4  ...   
                        Women+                    9902.2  ...   
           Saskatchewan Men+                       324.0  ...   
                        Total - Gender             606.4  ...   
                        Women+                     282.4  ...   

Data type                                     Unadjusted                    \
Age group                                 25 to 54 years 55 years and ove

### Transpose

In [ ]:
# transpose the data to make it easier to view
# This is similar to beyond 20/20 format
df.transpose()

REF_DATE                                                                                2019-01-01  \
Data type           GEO          Gender Age group         Labour force characteristics               
Seasonally adjusted Canada       Men+   15 to 24 years    Employment                        1332.8   
                                                          Employment rate                     58.3   
                                                          Full-time employment               782.6   
                                                          Labour force                      1502.6   
                                                          Part-time employment               550.2   
...                                                                                            ...   
Unadjusted          Saskatchewan Women+ 55 years and over Part-time employment                19.0   
                                                          Participation rate                  35.9   
                                                          Population                         162.5   
                                                          Unemployment                         2.2   
                                                          Unemployment rate                    3.8   

REF_DATE                                                                                2019-02-01  \
Data type           GEO          Gender Age group         Labour force characteristics               
Seasonally adjusted Canada       Men+   15 to 24 years    Employment                        1332.1   
                                                          Employment rate                     58.3   
                                                          Full-time employment               783.3   
                                                          Labour force                      1513.3   
                                                          Part-time employment               548.9   
...                                                                                            ...   
Unadjusted          Saskatchewan Women+ 55 years and over Part-time employment                18.9   
                                                          Participation rate                  35.4   
                                                          Population                         162.7   
                                                          Unemployment                         2.0   
                                                          Unemployment rate                    3.5   

REF_DATE                                                                                2019-03-01  \
Data type           GEO          Gender Age group         Labour force characteristics               
Seasonally adjusted Canada       Men+   15 to 24 years    Employment                        1323.1   
                                                          Employment rate                     57.9   
                                                          Full-time employment               771.9   
                                                          Labour force                      1506.0   
                                                          Part-time employment               551.1   
...                                                                                            ...   
Unadjusted          Saskatchewan Women+ 55 years and over Part-time employment                19.3   
                                                          Participation rate                  36.8   
                                                          Population                         163.0   
                                                          Unemployment                         2.3   
                                                          Unemployment rate                    3.8   

REF_DATE           

## Selecting Data

### Selecting by loc

In [77]:
# select rows by index label
df.loc['2023':'2024']

Data type                    Seasonally adjusted                  \
GEO                                       Canada                   
Gender                                      Men+                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE                                                           
2023-01-01                                1339.3            57.9   
2023-02-01                                1349.2            58.1   
2023-03-01                                1366.5            58.6   
2023-04-01                                1370.0            58.5   
2023-05-01                                1342.1            57.1   
2023-06-01                                1364.8            57.8   
2023-07-01                                1383.1            58.3   
2023-08-01                                1364.7            57.2   
2023-09-01                                1364.4            56.8   
2023-10-01                                1368.0            56.7   
2023-11-01                                1374.7            56.6   
2023-12-01                                1364.1            55.8   
2024-01-01                                1371.5            55.7   
2024-02-01                                1378.3            55.7   
2024-03-01                                1354.6            54.4   
2024-04-01                                1395.1            55.7   
2024-05-01                                1354.9            53.7   
2024-06-01                                1352.4            53.3   
2024-07-01                                1346.5            52.7   
2024-08-01                                1355.4            52.8   
2024-09-01                                1368.0            53.1   
2024-10-01                                1379.2            53.4   
2024-11-01                                1379.7            53.3   
2024-12-01                                1375.1            53.0   

Data type                                                       \
GEO                                                              
Gender                                                           
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE                                                         
2023-01-01                                  763.5       1500.3   
2023-02-01                                  777.5       1514.8   
2023-03-01                                  786.3       1517.4   
2023-04-01                                  785.0       1533.9   
2023-05-01                                  777.7       1518.6   
2023-06-01                                  779.3       1549.6   
2023-07-01                                  797.4       1544.9   
2023-08-01                                  804.7       1554.0   
2023-09-01                                  783.9       1540.1   
2023-10-01                                  774.0       1552.7   
2023-11-01                                  781.2       1561.5   
2023-12-01                                  780.1       1557.0   
2024-01-01                                  781.0       1545.4   
2024-02-01                                  795.8       1574.3   
2024-03-01                                  784.3       1569.2   
2024-04-01                                  827.9       1607.6   
2024-05-01                                  760.4       1572.7   
2024-06-01                                  793.9       1588.8   
2024-07-01                                  781.2       1600.1   
2024-08-01                                  769.3       1613.5   
2024-09-01                                  780.0       1615.2   
2024-10-01                                  795.5       1615.8   
2024-11-01                                  797.6       1635.3   
2024-12-01                                  8

In [79]:
# suubset the data on rows and a single column
df.loc['2024', ('Seasonally adjusted', 'Canada','Total - Gender', 
                '15 years and over','Employment')]

REF_DATE
2024-01-01    20577.1
2024-02-01    20607.7
2024-03-01    20614.5
2024-04-01    20700.5
2024-05-01    20698.3
2024-06-01    20715.9
2024-07-01    20712.9
2024-08-01    20742.6
2024-09-01    20779.3
2024-10-01    20782.6
2024-11-01    20826.4
2024-12-01    20917.4
Name: (Seasonally adjusted, Canada, Total - Gender, 15 years and over, Employment), dtype: float64

In [ ]:
# Header helps to know the selected time series
df.loc['2024', [('Seasonally adjusted', 'Canada', 'Employment',
                'Men+', '15 years and over')]]

Data type,Seasonally adjusted
GEO,Canada
Labour force characteristics,Employment
Gender,Men+
Age group,15 years and over
REF_DATE,
2024-01-01,10857.7
2024-02-01,10876.6
2024-03-01,10867.5
2024-04-01,10944.0
2024-05-01,10921.6


In [57]:
# Selecting two columms
df.loc['2024', [('Seasonally adjusted', 'Canada', 'Employment',
                'Men+', '15 years and over'),
                ('Seasonally adjusted', 'Canada', 'Employment',
                'Women+', '15 years and over')]]

Data type                    Seasonally adjusted                  
GEO                                       Canada                  
Labour force characteristics          Employment                  
Gender                                      Men+            Women+
Age group                      15 years and over 15 years and over
REF_DATE                                                          
2024-01-01                               10857.7            9719.4
2024-02-01                               10876.6            9731.1
2024-03-01                               10867.5            9747.0
2024-04-01                               10944.0            9756.5
2024-05-01                               10921.6            9776.7
2024-06-01                               10917.3            9798.6
2024-07-01                               10923.9            9789.0
2024-08-01                               10932.8            9809.8
2024-09-01                               10950.4            9828.9
2024-10-01                               10964.1            9818.5
2024-11-01                               10998.7            9827.7
2024-12-01                               11059.3            9858.0

In [65]:
# Another way to select columns

# create a set of conditions
# eaach condition can use bitwise operators to combine multiple conditions
cond_1 = df.columns.get_level_values('GEO') == 'Canada'
cond_2 = (df.columns.get_level_values('Labour force characteristics') == 'Employment') | \
(df.columns.get_level_values('Labour force characteristics') == 'Unemployment')
cond_3 = df.columns.get_level_values('Data type') == 'Unadjusted'
cond_4 = df.columns.get_level_values('Age group') == '15 years and over'
cond_5 = df.columns.get_level_values('Gender') == 'Total - Gender'

#use loc to select the data
df.loc['2024', cond_1 & cond_2 & cond_3 & cond_4 & cond_5]


Data type                           Unadjusted                  
GEO                                     Canada                  
Labour force characteristics        Employment      Unemployment
Gender                          Total - Gender    Total - Gender
Age group                    15 years and over 15 years and over
REF_DATE                                                        
2024-01-01                             20246.6            1314.5
2024-02-01                             20369.9            1309.7
2024-03-01                             20411.5            1407.6
2024-04-01                             20583.5            1370.4
2024-05-01                             20925.4            1399.6
2024-06-01                             21048.5            1383.4
2024-07-01                             20915.6            1505.9
2024-08-01                             20887.4            1682.5
2024-09-01                             20844.0            1334.2
2024-10-01                             20861.8            1327.7
2024-11-01                             20828.6            1445.3
2024-12-01                             20843.6            1369.7

### Selecting by xs

In [81]:
# cross-sections can be used to select data at a specific level of a MultiIndex
# one value for each level
df.xs(level='GEO', key='Canada', axis=1).tail()

Data type                    Seasonally adjusted                  \
Gender                                      Men+                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE                                                           
2024-09-01                                1368.0            53.1   
2024-10-01                                1379.2            53.4   
2024-11-01                                1379.7            53.3   
2024-12-01                                1375.1            53.0   
2025-01-01                                1399.7            53.8   

Data type                                                       \
Gender                                                           
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE                                                         
2024-09-01                                  780.0       1615.2   
2024-10-01                                  795.5       1615.8   
2024-11-01                                  797.6       1635.3   
2024-12-01                                  800.7       1634.9   
2025-01-01                                  831.6       1637.7   

Data type                                                             \
Gender                                                                 
Age group                                                              
Labour force characteristics Part-time employment Participation rate   
REF_DATE                                                               
2024-09-01                                  588.0               62.7   
2024-10-01                                  583.7               62.5   
2024-11-01                                  582.0               63.1   
2024-12-01                                  574.5               63.0   
2025-01-01                                  568.2               63.0   

Data type                                                               \
Gender                                                                   
Age group                                                                
Labour force characteristics Population Unemployment Unemployment rate   
REF_DATE                                                                 
2024-09-01                       2577.4        247.2              15.3   
2024-10-01                       2584.8        236.6              14.6   
2024-11-01                       2590.2        255.6              15.6   
2024-12-01                       2595.4        259.8              15.9   
2025-01-01                       2599.4        237.9              14.5   

Data type                                       ...        Unadjusted  \
Gender                                          ...            Women+   
Age group                    15 years and over  ...    25 to 54 years   
Labour force characteristics        Employment  ... Unemployment rate   
REF_DATE                                        ...                     
2024-09-01                             10950.4  ...               5.1   
2024-10-01                             10964.1  ...               5.3   
2024-11-01                             10998.7  ...               5.4   
2024-12-01                             11059.3  ...               4.9   
2025-01-01                             11091.2  ...               5.5   

Data type                                                       \
Gender                                                           
Age group                    55 years and over                   
Labour force characteristics        Employment Employment rate   
REF_DATE                                                         
2024-09-01                              1973.5            29.9   
2024-10-01                              1971.3            29.8   
202

In [82]:
# we can filter on multiple levels
df.xs(level=('GEO', 
'Labour force characteristics'), key=('Canada', 'Employment'), axis=1)

Data type  Seasonally adjusted                                   \
Gender                    Men+                                    
Age group       15 to 24 years 15 years and over 25 to 54 years   
REF_DATE                                                          
2019-01-01              1332.8            9956.3         6369.9   
2019-02-01              1332.1            9963.0         6369.8   
2019-03-01              1323.1            9969.4         6376.1   
2019-04-01              1363.0           10034.8         6377.5   
2019-05-01              1318.5           10026.4         6412.8   
...                        ...               ...            ...   
2024-09-01              1368.0           10950.4         7217.9   
2024-10-01              1379.2           10964.1         7214.6   
2024-11-01              1379.7           10998.7         7247.0   
2024-12-01              1375.1           11059.3         7277.4   
2025-01-01              1399.7           11091.2         7305.0   

Data type                                                                     \
Gender                       Total - Gender                                    
Age group  55 years and over 15 to 24 years 15 years and over 25 to 54 years   
REF_DATE                                                                       
2019-01-01            2253.7         2604.5           18920.5        12246.5   
2019-02-01            2261.0         2618.5           18965.7        12269.0   
2019-03-01            2270.2         2598.6           18932.9        12234.6   
2019-04-01            2294.3         2660.9           19053.7        12263.3   
2019-05-01            2295.2         2617.6           19067.3        12311.6   
...                      ...            ...               ...            ...   
2024-09-01            2364.5         2686.2           20779.3        13765.5   
2024-10-01            2370.4         2704.4           20782.6        13759.8   
2024-11-01            2372.0         2700.8           20826.4        13801.4   
2024-12-01            2406.8         2693.3           20917.4        13842.4   
2025-01-01            2386.5         2723.8           20993.4        13905.7   

Data type                                                      ...  \
Gender                               Women+                    ...   
Age group  55 years and over 15 to 24 years 15 years and over  ...   
REF_DATE                                                       ...   
2019-01-01            4069.5         1271.8            8964.2  ...   
2019-02-01            4078.2         1286.3            9002.7  ...   
2019-03-01            4099.7         1275.6            8963.5  ...   
2019-04-01            4129.6         1297.9            9019.0  ...   
2019-05-01            4138.0         1299.2            9040.9  ...   
...                      ...            ...               ...  ...   
2024-09-01            4327.6         1318.2            9828.9  ...   
2024-10-01            4318.4         1325.3            9818.5  ...   
2024-11-01            4324.2         1321.1            9827.7  ...   
2024-12-01            4381.7         1318.2            9858.0  ...   
2025-01-01            4364.0         1324.0            9902.2  ...   

Data type      Unadjusted                                                     \
Gender               Men+                   Total - Gender                     
Age group  25 to 54 years 55 years and over 15 to 24 years 15 years and over   
REF_DATE                                                                       
2019-01-01         6268.3            2208.6         2427.6           18579.8   
2019-02-01         6278.7            2224.9         2457.0           18673.8   
2019-03-01         6293.9            2246.5         2459.1           18674.2   
2019-04-01         6328.0            2275.2         2521.9           18875.6   
2019-05-01         6445.1            2307.7         2712.0           19253.1   
...                   